<a href="https://colab.research.google.com/github/DavidBarral11/Mineria_de_datos_USAC/blob/main/REDES_NEURONALES_FALTAS_JUDICIALES_ESTADO_EBRIEDAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split



Saving base-faltas-judiciales-2024.xlsx to base-faltas-judiciales-2024.xlsx


In [2]:

filename = list(uploaded.keys())[0]
print("Archivo cargado:", filename)

data = pd.read_excel(filename)

data.head()


Archivo cargado: base-faltas-judiciales-2024.xlsx


,num_corre,depto_boleta,muni_boleta,mes_boleta,año_boleta,falta_inf,sexo_inf,edad_inf,grupo_etnico_inf,est_conyugal_inf,...,g_primarios,area_geo_inf,est_ebriedad_inf,g_edad_80ymas,g_edad_60ymas,edad_quinquenales,depto_nacimiento_inf,nacionalidad_inf,subg_principales,gran_grupos
0,1,1,102,4,2024,4,1,22,2,1,...,9621,2,2,3,3,5,1,1,96,9
1,2,1,102,4,2024,3,1,20,2,1,...,8322,1,1,3,3,5,1,1,83,8
2,3,1,102,4,2024,3,1,18,2,1,...,9998,1,1,2,2,4,1,1,98,11
3,4,1,102,4,2024,4,1,18,2,1,...,9998,1,1,2,2,4,1,1,98,11
4,5,1,102,4,2024,3,1,21,2,1,...,7122,1,1,3,3,5,1,1,71,7


In [3]:
data['edad_inf'] = data['edad_inf'].astype(str)
mask_9s = data['edad_inf'].str.fullmatch(r'9+')
data.loc[mask_9s, 'edad_inf_limpia'] = np.nan
data.loc[~mask_9s, 'edad_inf_limpia'] = pd.to_numeric(data.loc[~mask_9s, 'edad_inf'], errors='coerce')

data['est_ebriedad_inf'] = pd.to_numeric(data['est_ebriedad_inf'], errors='coerce')

data = data.dropna(subset=['est_ebriedad_inf'])

data['es_ebrio'] = data['est_ebriedad_inf'].apply(lambda x: 1 if x == 1 else 0)

data = data.sample(frac=1, random_state=42).reset_index(drop=True)


In [4]:
X = data[[
    'sexo_inf',
    'edad_inf_limpia',
    'grupo_etnico_inf',
    'est_conyugal_inf',
    'cond_alfabetismo_inf',
    'niv_escolaridad_inf',
    'area_geo_inf',
    'falta_inf',
    'gran_grupos'
]]

X = X.fillna(0)

y = data['es_ebrio']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [5]:
model = Sequential()
model.add(Dense(16, input_dim=9, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 305 (1.19 KB)

 Trainable params: 305 (1.19 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_test, y_test),
    verbose=1
)


Epoch 1/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5654 - loss: 0.8298 - val_accuracy: 0.6698 - val_loss: 0.6179
Epoch 2/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7242 - loss: 0.5865 - val_accuracy: 0.7579 - val_loss: 0.5495
Epoch 3/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7701 - loss: 0.5351 - val_accuracy: 0.7660 - val_loss: 0.5293
Epoch 4/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7777 - loss: 0.5190 - val_accuracy: 0.7841 - val_loss: 0.5113
Epoch 5/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7876 - loss: 0.5079 - val_accuracy: 0.7926 - val_loss: 0.4996
Epoch 6/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7871 - loss: 0.5068 - val_accuracy: 0.7973 - val_loss: 0.4925
Epoch 7/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8015 - loss: 0.4811 - val_accuracy: 0.7967 - val_loss: 0.4869
Epoch 8/100
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7897 - loss: 0.4911 - val_accu

In [8]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')




Loss: 0.4360, Accuracy: 0.8046


In [9]:
hecho = np.array([[1, 30, 1, 2, 1, 3, 1, 2, 1]], dtype='float32')

res = model.predict(hecho)
print("Probabilidad estimada de que el infractor esté ebrio:", res)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Probabilidad estimada de que el infractor esté ebrio: [[0.8502049]]
